<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/PyTorch_Blocks_and_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import torch
from torch import nn
from torch.nn import functional as F

Some part of the code is based on d2l.ai book: http://d2l.ai/

In [128]:
### basing on the d2l.ai book 

In [129]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))

X=torch.rand(2,20)
net(X)

tensor([[-0.2511, -0.1936, -0.0241,  0.0806,  0.4070, -0.0377, -0.1729,  0.2600,
          0.0812, -0.1201],
        [-0.0955, -0.2428,  0.0699, -0.0722,  0.2934,  0.1410, -0.0233,  0.1487,
         -0.0375,  0.0093]], grad_fn=<AddmmBackward>)

In [130]:
net.forward(X)

tensor([[-0.2511, -0.1936, -0.0241,  0.0806,  0.4070, -0.0377, -0.1729,  0.2600,
          0.0812, -0.1201],
        [-0.0955, -0.2428,  0.0699, -0.0722,  0.2934,  0.1410, -0.0233,  0.1487,
         -0.0375,  0.0093]], grad_fn=<AddmmBackward>)

In [131]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


In [132]:
net = MLP()
net(X)

tensor([[ 0.0736, -0.0314, -0.2261, -0.1756,  0.2326, -0.0241, -0.0191, -0.2589,
          0.0304,  0.2953],
        [-0.0896, -0.0673, -0.0865, -0.2184,  0.1027, -0.0237,  0.1227, -0.1215,
         -0.0883,  0.1061]], grad_fn=<AddmmBackward>)

In [133]:
### Custom Sequential Block

In [134]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X



In [135]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1326, -0.1398, -0.0072, -0.1961, -0.1421,  0.2422,  0.0093, -0.2049,
         -0.1839, -0.2150],
        [ 0.0554, -0.0163,  0.1596, -0.1814, -0.1092,  0.0948, -0.0435, -0.0639,
         -0.1689, -0.1130]], grad_fn=<AddmmBackward>)

MyParallel model: several models use the same input, executed in paralle and the results of the models are concatenated in one output.

In [136]:
class MyParallel(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        list_res=[]
        for block in self._modules.values():
          list_res.append(block(X))
        concat_fin=torch.cat(list_res, dim = -1)
        return concat_fin



In [137]:
list_models = [nn.Linear(20,20),nn.Linear(20,20),nn.Linear(20,20)]

In [138]:
net = MyParallel(nn.Linear(20,20),nn.Linear(20,20),nn.Linear(20,20))
#X = torch.tensor([1,10])
net(X)

tensor([[-0.8378,  0.4368, -0.0836, -0.1544,  0.2442, -0.2566,  0.2811,  0.1476,
         -0.0117, -0.5526, -0.5293,  0.4282, -0.3261,  0.0572,  0.1135, -0.0472,
          0.4304,  0.0641,  0.1270, -0.2490, -0.5219,  0.1622, -0.3052, -0.1004,
         -0.3826,  0.2613, -0.1517, -0.1587,  0.0329, -0.6025, -0.1988,  0.5530,
         -0.0732, -0.1856, -0.4693,  0.2454,  0.4006,  0.2873, -0.0250,  0.0034,
          0.5087, -0.5172,  0.1716, -0.0663,  0.0525, -0.4478,  0.1103,  0.6191,
         -0.2156,  0.2591,  0.7326,  0.4514, -0.3594, -0.0020,  0.1164, -0.2802,
          0.2221, -0.6473,  0.0984,  0.0911],
        [-0.9652,  0.7165, -0.0055,  0.2759,  0.2285, -0.0468,  0.5170,  0.1011,
          0.3189, -0.4060, -0.3416,  0.4551, -0.1653,  0.0941, -0.3136,  0.0370,
         -0.3652, -0.3999,  0.1609,  0.1851, -0.8391,  0.4178,  0.3077, -0.1437,
         -0.0901,  0.6757, -0.1040, -0.1135, -0.2441, -0.3089, -0.0906,  0.3661,
          0.0928,  0.1847, -0.2205,  0.1841,  0.0726,  0.1751, 

In [139]:
net = MyParallel(*list_models)
net(X)

tensor([[-0.1488, -0.1264,  0.4381,  0.7137, -0.6443, -0.0645,  0.3388, -0.1104,
          0.1917, -0.1900,  0.0039, -0.0677,  0.3395,  0.4270, -0.1230,  0.1560,
         -0.8479,  0.0581, -0.2496, -0.0094,  0.8846,  0.1550, -0.0349,  0.2033,
          0.0729, -0.1458,  0.0112, -0.8391, -0.4942,  0.6476,  0.1052,  0.2037,
          0.2804,  0.0924,  0.3138, -0.0055,  0.3365,  0.0590,  0.2191,  0.1527,
         -0.4742,  0.3799, -0.2444, -0.1667,  0.6810,  0.2701,  0.3883, -0.0997,
          0.5334,  0.3969, -0.2636, -0.3816,  0.3145, -0.1896,  0.0775,  0.1546,
          0.2503,  0.0388,  0.0965, -0.2552],
        [-0.3358,  0.2262,  0.6327,  0.4836, -0.1902,  0.3506,  0.4452, -0.2592,
         -0.2986, -0.1265, -0.3226,  0.0709,  0.3142, -0.1864,  0.1012, -0.0133,
         -0.2613,  0.1467, -0.3816,  0.2484,  0.8834,  0.1443, -0.3368,  0.0731,
          0.0561, -0.6898, -0.0343, -0.6978, -0.3101,  0.3910, -0.1583, -0.1757,
         -0.0543,  0.0587,  0.0215,  0.1356,  0.3149, -0.0574, 

In [145]:
##### Nesting Modules
class NestedBlocks(nn.Module):
  def __init__(self,*args):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20,20),nn.Linear(20,20))
    self.linear = nn.Linear(20,20)

  def forward(self, X):
    return self.linear(self.net(X))



In [146]:
tensor = torch.ones([2,20])
net = NestedBlocks()
net(X)

tensor([[ 0.0190, -0.1838,  0.1627,  0.2060, -0.1366, -0.0127, -0.2582,  0.1960,
         -0.2671, -0.0659,  0.0070,  0.0171, -0.0207, -0.2459, -0.2035,  0.0998,
         -0.3006, -0.1006,  0.0776,  0.0604],
        [ 0.2123, -0.0293,  0.0945,  0.1446, -0.0638, -0.0717, -0.0815, -0.0664,
         -0.2449, -0.1005, -0.0660,  0.0515,  0.1943, -0.2329, -0.1916,  0.0756,
         -0.1726, -0.2042, -0.2117,  0.0126]], grad_fn=<AddmmBackward>)